In [ ]:
!pip install ibapi polars lxml sqlitedict
!pip list

In [ ]:
import logging
import polars as pl
import locale
import os

from utils import SetupLogger, ibConnect, getAccounts, getOpenOrders, getCurrencies, getAssetDetails, ibDisconnect, computeThings
 

logger = SetupLogger()
logging.getLogger().setLevel(logging.WARNING)

ipaddr = os.getenv("IPADDR", "127.0.0.1")
BaseCur = ['USD', 'CHF', 'EUR'] 

app = ibConnect(ipaddr)

if app is not None:
    getAccounts(app)
    getOpenOrders(app)
    getCurrencies(app, BaseCur)
    getAssetDetails(app)

    ibDisconnect(app)

    computeThings(app, BaseCur)
    
    for account in app.accounts:
        df = pl.DataFrame(app.portfolios[account])
        with pl.Config(
            tbl_cell_numeric_alignment="RIGHT",
            thousands_separator="",
            decimal_separator=locale.localeconv()["decimal_point"],
            float_precision=4,
            set_tbl_column_data_type_inline=False,
        ):
            display(df) 
            df.write_csv(file=f"{account}.csv", separator='\t', float_precision=4, null_value="")
